# Retrieve data and load into database

- Extract emission related indicators from World Bank database using WBGAPI
- Clean and prepare data for database
- Load data into database

In [1]:
# Import dependencies
import pandas as pd
import numpy as np

import wbgapi as wb

from sqlalchemy import create_engine
import psycopg2
from config import db_password

%matplotlib inline

import matplotlib.pyplot as plt


## Series Indicators from World Bank Database

#### Emission Indicators:
- EN.ATM.CO2E.KT - CO2 emissions (kt) (emissions_total)
- EN.ATM.CO2E.PC - CO2 emissions (metric tons per capita) (emissions_per_capita)
- EN.ATM.CO2E.PP.GD - CO2 emissions (kg per PPP dollar of GDP) (emissions_per_gdp)
- EN.ATM.NOXE.KT.CE - Nitrous Oxide emissions
- EN.ATM.METH.KT.CE - Methane emissions

#### Energy Indicators:
- EG.USE.PCAP.KG.OE - Energy Use (kg of oil equivalent per capita) (energy_use_per_capita)
- AG.YLD.CREL.KG - Cereal yield (cereal_yield)
- EG.ELC.ACCS.ZS - Access to Electricity (% of population) (electricity_access_percent)
- EG.USE.COMM.FO.ZS - Fossil fuel energy consumption (% of total) (fossil_energy_use)
- EG.FEC.RNEW.ZS - Renewable energy consumption (% of total final energy consumption)
    
#### Economic Indicators:
- NY.GDP.PCAP.CD - GDP per capita (current USD) (gdp_per_capita)

#### Population Indicators:
- SP.POP.TOTL - Total Population (pop_total)
- SP.POP.GROW - Population growth (annual %) (pop_growth_percent)
- SP.URB.TOTL - Urban population (urban_pop_total)
- SP.URB.TOTL.IN.ZS - Urban population (% of total population) (urban_pop_percent)

- SP.POP.1564.TO.ZS - Population ages 15-64 (% of total population) (pop_1564_percent)
- SP.POP.65UP.TO.ZS - Population ages 65 and above (% of total population) (pop_65up_percent)
- SP.POP.0014.TO.ZS - Population ages 0-14 (% of total population) (pop_0014_percent)

#### Other
- AG.LND.FRST.ZS - Forest area (%)

#### Countries
- Majority of countries available in World Bank database.

#### Years
- 1990-2017

## Extract indicator data using WBGAPI

In [2]:
indicator_list = [
    'EN.ATM.CO2E.KT',
    'EN.ATM.CO2E.PC',
    'EN.ATM.CO2E.PP.GD',
    'EG.USE.PCAP.KG.OE',
    'AG.YLD.CREL.KG',
    'EG.ELC.ACCS.ZS',
    'EG.USE.COMM.FO.ZS',
    'NY.GDP.PCAP.CD',
    'SP.POP.TOTL',
    'SP.POP.GROW',
    'SP.URB.TOTL',
    'SP.URB.TOTL.IN.ZS',
    'SP.POP.1564.TO.ZS',
    'SP.POP.65UP.TO.ZS',
    'SP.POP.0014.TO.ZS',
    'EG.FEC.RNEW.ZS',
    'AG.LND.FRST.ZS',
    'EN.ATM.NOXE.KT.CE',
    'EN.ATM.METH.KT.CE'
]

In [3]:
# Variables to hold year range
year_start = 1990
year_end = 2017

In [4]:
# Make an API call and load data into a Pandas data frame #apprx. 5mins to load
raw_df = wb.data.DataFrame(indicator_list, time=range(year_start, year_end), numericTimeKeys=True, labels=True, columns='series').reset_index()
raw_df.head()

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.COMM.FO.ZS,EG.USE.PCAP.KG.OE,...,EN.ATM.METH.KT.CE,EN.ATM.NOXE.KT.CE,NY.GDP.PCAP.CD,SP.POP.0014.TO.ZS,SP.POP.1564.TO.ZS,SP.POP.65UP.TO.ZS,SP.POP.GROW,SP.POP.TOTL,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
0,ZWE,2016,Zimbabwe,2016,45.570273,435.1,42.561729,81.90,NaN,NaN,...,11869.999886,4980.0,1421.787789,42.517883,54.314703,3.167414,2.081806,14452704.0,4667645.0,32.296
1,ZWE,2015,Zimbabwe,2015,45.689363,557.5,33.700001,80.82,NaN,NaN,...,12260.000229,5400.0,1410.329174,42.872671,54.029704,3.097626,2.136294,14154937.0,4584076.0,32.385
2,ZWE,2014,Zimbabwe,2014,45.808453,831.4,32.299999,80.27,NaN,NaN,...,11369.999886,4910.0,1407.034293,43.172363,53.799451,3.028186,2.191391,13855753.0,4503674.0,32.504
3,ZWE,2013,Zimbabwe,2013,45.927543,668.5,40.498375,78.87,29.095759,832.572236,...,12340.000153,5500.0,1408.367810,43.370440,53.634994,2.994566,2.163267,13555422.0,4426387.0,32.654
4,ZWE,2012,Zimbabwe,2012,46.046633,695.7,44.000000,77.50,28.860873,814.910235,...,12460.000038,5800.0,1290.193956,43.453948,53.556956,2.989096,1.822309,13265331.0,4355539.0,32.834


### Overview of retrieved data

In [5]:
# Rows and columns of data set
raw_df.shape

(7182, 23)

In [6]:
# Columns in the dataset
raw_df.columns

Index(['economy', 'time', 'Country', 'Time', 'AG.LND.FRST.ZS',
       'AG.YLD.CREL.KG', 'EG.ELC.ACCS.ZS', 'EG.FEC.RNEW.ZS',
       'EG.USE.COMM.FO.ZS', 'EG.USE.PCAP.KG.OE', 'EN.ATM.CO2E.KT',
       'EN.ATM.CO2E.PC', 'EN.ATM.CO2E.PP.GD', 'EN.ATM.METH.KT.CE',
       'EN.ATM.NOXE.KT.CE', 'NY.GDP.PCAP.CD', 'SP.POP.0014.TO.ZS',
       'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS', 'SP.POP.GROW', 'SP.POP.TOTL',
       'SP.URB.TOTL', 'SP.URB.TOTL.IN.ZS'],
      dtype='object')

In [7]:
# Datatypes of columns
raw_df.dtypes

economy               object
time                   int64
Country               object
Time                  object
AG.LND.FRST.ZS       float64
AG.YLD.CREL.KG       float64
EG.ELC.ACCS.ZS       float64
EG.FEC.RNEW.ZS       float64
EG.USE.COMM.FO.ZS    float64
EG.USE.PCAP.KG.OE    float64
EN.ATM.CO2E.KT       float64
EN.ATM.CO2E.PC       float64
EN.ATM.CO2E.PP.GD    float64
EN.ATM.METH.KT.CE    float64
EN.ATM.NOXE.KT.CE    float64
NY.GDP.PCAP.CD       float64
SP.POP.0014.TO.ZS    float64
SP.POP.1564.TO.ZS    float64
SP.POP.65UP.TO.ZS    float64
SP.POP.GROW          float64
SP.POP.TOTL          float64
SP.URB.TOTL          float64
SP.URB.TOTL.IN.ZS    float64
dtype: object

In [8]:
# Descriptive statistics
raw_df.describe()

,time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.COMM.FO.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD,EN.ATM.METH.KT.CE,EN.ATM.NOXE.KT.CE,NY.GDP.PCAP.CD,SP.POP.0014.TO.ZS,SP.POP.1564.TO.ZS,SP.POP.65UP.TO.ZS,SP.POP.GROW,SP.POP.TOTL,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
count,7182.000000,6857.000000,5948.000000,6041.000000,6876.000000,4723.000000,4751.000000,6.430000e+03,6430.000000,6043.000000,6.453000e+03,6.426000e+03,6687.000000,7155.000000,7155.000000,7155.000000,7153.000000,7.155000e+03,7.101000e+03,7101.000000
mean,2003.000000,32.654166,2868.612911,79.730564,31.238203,65.910120,2270.160374,9.809105e+05,4.212636,0.351621,3.070970e+05,1.106342e+05,10971.471653,31.129357,61.741906,7.128736,1.489059,2.564654e+08,1.194758e+08,55.109050
std,7.789423,23.406545,2292.659189,29.639967,30.182347,29.004060,2669.532102,3.162623e+06,5.252726,0.300849,8.898596e+05,3.098570e+05,18708.929894,10.559871,6.947037,4.953521,1.662774,8.138013e+08,3.752095e+08,23.541963
min,1990.000000,0.000000,34.300000,0.533899,0.000000,0.000000,9.579196,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,22.850371,11.599104,46.097007,0.171770,-27.722225,9.182000e+03,3.733000e+03,5.416000
25%,1996.000000,12.530100,1397.618511,65.573410,4.761719,45.438606,603.049194,2.210000e+03,0.631896,0.174115,3.300000e+03,9.800000e+02,980.249123,21.243883,55.403983,3.302148,0.554488,1.322634e+06,6.428460e+05,35.175000
50%,2003.000000,30.874170,2371.890669,98.240425,20.992775,75.875796,1238.114597,2.315000e+04,2.403985,0.276880,1.376000e+04,5.390000e+03,3197.331024,30.983912,63.224729,5.106476,1.420004,8.417823e+06,4.009752e+06,53.979000
75%,2010.000000,47.761159,3760.575000,100.000000,55.007769,88.652917,3025.736971,2.441525e+05,6.228607,0.413474,8.551000e+04,3.253000e+04,12807.635420,41.144452,67.099729,10.479762,2.424812,5.547069e+07,3.165639e+07,74.127000
max,2016.000000,98.574551,36761.900000,100.000000,98.340000,100.000000,21420.628504,3.319873e+07,47.651306,2.382059,7.948160e+06,2.920660e+06,203266.913745,51.180881,86.079246,35.041340,19.360429,7.491934e+09,4.066385e+09,100.000000


## Clean and Prepare data for database

- Rename columns 
- Create country dataframe
- Remove unnecessary columns

#####  Missing values and filter data - This step is done in different file as we are using same data for both visualization and machine learning. 

In [9]:
# Assign raw dataframe to another that can be modified
df_data_clean = raw_df.copy()
df_data_clean.sample(5)

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.COMM.FO.ZS,EG.USE.PCAP.KG.OE,...,EN.ATM.METH.KT.CE,EN.ATM.NOXE.KT.CE,NY.GDP.PCAP.CD,SP.POP.0014.TO.ZS,SP.POP.1564.TO.ZS,SP.POP.65UP.TO.ZS,SP.POP.GROW,SP.POP.TOTL,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
1820,PAK,2005,Pakistan,2005,5.581277,2615.8,70.512901,47.960000,60.080801,438.783732,...,108629.997253,46380.0,688.500588,41.094946,55.357716,3.547338,2.158492,174372098.0,59255126.0,33.982
250,URY,2009,Uruguay,2009,9.685007,3884.9,99.099998,44.640000,60.649090,1235.017146,...,22409.999847,7940.0,9467.534193,22.604612,63.146112,14.249275,0.240409,3344156.0,3150496.0,94.209
2114,NPL,2008,Nepal,2008,41.337817,2356.2,56.361980,90.470000,9.886058,358.375174,...,21000.000000,5200.0,466.693379,37.438445,58.100152,4.461403,0.626510,26881544.0,4329273.0,16.105
5851,AFG,1997,Afghanistan,1997,1.852782,1348.8,NaN,41.432601,NaN,NaN,...,9399.999619,3430.0,NaN,49.001634,48.711896,2.286470,3.910605,17788819.0,3878852.0,21.805
2125,NPL,1997,Nepal,1997,40.196727,1946.6,18.132868,90.708660,9.655924,305.182405,...,18030.000687,4690.0,211.561947,41.205000,55.130272,3.664728,2.022288,23249417.0,2751336.0,11.834


#### Rename the columns

In [10]:
# Rename the columns headers to meaningful names
column_names = {
    'economy':'country_code',
    'Country':'country_name',
    'EN.ATM.CO2E.KT':'emissions_total',
    'EN.ATM.CO2E.PC':'emissions_per_capita',
    'EN.ATM.CO2E.PP.GD':'emissions_per_gdp',
    'EG.USE.PCAP.KG.OE':'energy_use_per_capita',
    'AG.YLD.CREL.KG':'cereal_yield',
    'EG.ELC.ACCS.ZS':'electricity_access_percent',
    'EG.USE.COMM.FO.ZS':'fossil_energy_use',
    'NY.GDP.PCAP.CD':'gdp_per_capita',
    'SP.POP.TOTL':'pop_total',
    'SP.POP.GROW':'pop_growth_percent',
    'SP.URB.TOTL':'urban_pop_total',
    'SP.URB.TOTL.IN.ZS':'urban_pop_percent',
    'SP.POP.1564.TO.ZS':'pop_1564_percent',
    'SP.POP.65UP.TO.ZS':'pop_65up_percent',
    'SP.POP.0014.TO.ZS':'pop_0014_percent',
    'EG.FEC.RNEW.ZS' : 'renewable_energy_consumption',
    'AG.LND.FRST.ZS': 'forest_area_in_per',
    'EN.ATM.NOXE.KT.CE':'nitrous_oxide_emissions(mt_CO2_emissions)',
    'EN.ATM.METH.KT.CE':'methane_emissions(kt_CO2_emissions)',
    'time':'year'
}

In [11]:
# Rename the columns headers
df_data_clean = df_data_clean.rename(columns=column_names)
df_data_clean.sample(5)

,country_code,year,country_name,Time,forest_area_in_per,cereal_yield,electricity_access_percent,renewable_energy_consumption,fossil_energy_use,energy_use_per_capita,...,methane_emissions(kt_CO2_emissions),nitrous_oxide_emissions(mt_CO2_emissions),gdp_per_capita,pop_0014_percent,pop_1564_percent,pop_65up_percent,pop_growth_percent,pop_total,urban_pop_total,urban_pop_percent
1033,SSD,2009,South Sudan,2009,NaN,NaN,3.000000,NaN,NaN,NaN,...,47659.999847,21710.0,1325.275077,43.682052,53.818438,2.499511,4.491271,9229227.0,1633665.0,17.701000
5263,BTN,1991,Bhutan,1991,53.863151,1027.400000,NaN,95.919938,NaN,NaN,...,569.999993,110.0,423.314713,41.820736,55.117490,3.061774,1.645996,567710.0,97425.0,17.161000
2749,LTU,1994,Lithuania,1994,31.509253,1753.400000,100.000000,9.823367,66.809295,2220.420361,...,5590.000153,3760.0,NaN,22.099592,65.942360,11.958048,-0.694004,3657144.0,2462904.0,67.345000
6642,IDB,2016,IDA blend,2016,30.030892,2073.519551,65.467867,NaN,NaN,NaN,...,411730.006777,207010.0,1818.404972,40.350938,56.216757,3.432305,1.931524,537358412.0,221601322.0,41.239016
405,TUV,2016,Tuvalu,2016,33.333333,NaN,97.300003,9.460000,NaN,NaN,...,10.000000,0.0,3836.112926,31.514928,63.011426,5.473645,-0.230107,10852.0,6581.0,60.645000


In [12]:
# Check for duplicate rows
df_data_clean.duplicated().sum()

0

In [13]:
df_data_clean.describe()

,year,forest_area_in_per,cereal_yield,electricity_access_percent,renewable_energy_consumption,fossil_energy_use,energy_use_per_capita,emissions_total,emissions_per_capita,emissions_per_gdp,methane_emissions(kt_CO2_emissions),nitrous_oxide_emissions(mt_CO2_emissions),gdp_per_capita,pop_0014_percent,pop_1564_percent,pop_65up_percent,pop_growth_percent,pop_total,urban_pop_total,urban_pop_percent
count,7182.000000,6857.000000,5948.000000,6041.000000,6876.000000,4723.000000,4751.000000,6.430000e+03,6430.000000,6043.000000,6.453000e+03,6.426000e+03,6687.000000,7155.000000,7155.000000,7155.000000,7153.000000,7.155000e+03,7.101000e+03,7101.000000
mean,2003.000000,32.654166,2868.612911,79.730564,31.238203,65.910120,2270.160374,9.809105e+05,4.212636,0.351621,3.070970e+05,1.106342e+05,10971.471653,31.129357,61.741906,7.128736,1.489059,2.564654e+08,1.194758e+08,55.109050
std,7.789423,23.406545,2292.659189,29.639967,30.182347,29.004060,2669.532102,3.162623e+06,5.252726,0.300849,8.898596e+05,3.098570e+05,18708.929894,10.559871,6.947037,4.953521,1.662774,8.138013e+08,3.752095e+08,23.541963
min,1990.000000,0.000000,34.300000,0.533899,0.000000,0.000000,9.579196,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,22.850371,11.599104,46.097007,0.171770,-27.722225,9.182000e+03,3.733000e+03,5.416000
25%,1996.000000,12.530100,1397.618511,65.573410,4.761719,45.438606,603.049194,2.210000e+03,0.631896,0.174115,3.300000e+03,9.800000e+02,980.249123,21.243883,55.403983,3.302148,0.554488,1.322634e+06,6.428460e+05,35.175000
50%,2003.000000,30.874170,2371.890669,98.240425,20.992775,75.875796,1238.114597,2.315000e+04,2.403985,0.276880,1.376000e+04,5.390000e+03,3197.331024,30.983912,63.224729,5.106476,1.420004,8.417823e+06,4.009752e+06,53.979000
75%,2010.000000,47.761159,3760.575000,100.000000,55.007769,88.652917,3025.736971,2.441525e+05,6.228607,0.413474,8.551000e+04,3.253000e+04,12807.635420,41.144452,67.099729,10.479762,2.424812,5.547069e+07,3.165639e+07,74.127000
max,2016.000000,98.574551,36761.900000,100.000000,98.340000,100.000000,21420.628504,3.319873e+07,47.651306,2.382059,7.948160e+06,2.920660e+06,203266.913745,51.180881,86.079246,35.041340,19.360429,7.491934e+09,4.066385e+09,100.000000


### Create country dataframe for country table

In [14]:
#creating new df to have country names
countries_df = df_data_clean.copy()
countries_df.head()

,country_code,year,country_name,Time,forest_area_in_per,cereal_yield,electricity_access_percent,renewable_energy_consumption,fossil_energy_use,energy_use_per_capita,...,methane_emissions(kt_CO2_emissions),nitrous_oxide_emissions(mt_CO2_emissions),gdp_per_capita,pop_0014_percent,pop_1564_percent,pop_65up_percent,pop_growth_percent,pop_total,urban_pop_total,urban_pop_percent
0,ZWE,2016,Zimbabwe,2016,45.570273,435.1,42.561729,81.90,NaN,NaN,...,11869.999886,4980.0,1421.787789,42.517883,54.314703,3.167414,2.081806,14452704.0,4667645.0,32.296
1,ZWE,2015,Zimbabwe,2015,45.689363,557.5,33.700001,80.82,NaN,NaN,...,12260.000229,5400.0,1410.329174,42.872671,54.029704,3.097626,2.136294,14154937.0,4584076.0,32.385
2,ZWE,2014,Zimbabwe,2014,45.808453,831.4,32.299999,80.27,NaN,NaN,...,11369.999886,4910.0,1407.034293,43.172363,53.799451,3.028186,2.191391,13855753.0,4503674.0,32.504
3,ZWE,2013,Zimbabwe,2013,45.927543,668.5,40.498375,78.87,29.095759,832.572236,...,12340.000153,5500.0,1408.367810,43.370440,53.634994,2.994566,2.163267,13555422.0,4426387.0,32.654
4,ZWE,2012,Zimbabwe,2012,46.046633,695.7,44.000000,77.50,28.860873,814.910235,...,12460.000038,5800.0,1290.193956,43.453948,53.556956,2.989096,1.822309,13265331.0,4355539.0,32.834


In [15]:
# Extract countries into new df
countries_df = countries_df.drop_duplicates(subset=['country_code'], keep='last')
countries_df = countries_df[['country_code','country_name']]
countries_df.reset_index(drop=True, inplace=True)
countries_df

,country_code,country_name
0,ZWE,Zimbabwe
1,ZMB,Zambia
2,YEM,"Yemen, Rep."
3,PSE,West Bank and Gaza
4,VIR,Virgin Islands (U.S.)
...,...,...
261,CEB,Central Europe and the Baltics
262,CSS,Caribbean small states
263,ARB,Arab World
264,AFW,Africa Western and Central


In [16]:
countries_df.shape

(266, 2)

#### Remove unnecessary columns

In [17]:
# Remove Time as it has duplicate column and country column as df has country code
df_data_clean.drop(columns = ['Time','country_name'], axis = 1, inplace = True)
df_data_clean.shape

(7182, 21)

## Save data into CSVs
Further cleaning of data has done in seperate files according to visualization and machine learning processes

In [18]:
#save emissions data and country data into csvs
df_data_clean.to_csv("../Resources/model_emissions.csv", index = False)
countries_df.to_csv("../Resources/country.csv", index = False)

## Load data into database
- Load countries with a SQL table name: country
- Load feature data with a SQl table name: ghg_emissions

In [19]:
# create database connection
user = "postgres"
password = db_password
host = "emissions-db.cr5mfnfivfxl.us-east-1.rds.amazonaws.com"
port = '5432'
database = "GHG_emissions"

conn_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(conn_string)

In [20]:
# Load dataframe into SQL tables
countries_df.to_sql('country', con=engine, index=False, if_exists='replace')
df_data_clean.to_sql('ghg_emissions', con=engine, index=False, if_exists='replace')